### Assignment 2
Below is an example that runs native CUDA code.

1.   We investigate the CUDA version, drivers and the avaiable GPU with nvidia-smi and nvcc-version
2.   We use the IPython magic command "%%writefile filename" to save a *.cu program
3.   We then compile and run the *.cu program with nvcc







In [1]:
!nvcc --version
!nvidia-smi

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2023 NVIDIA Corporation
Built on Tue_Aug_15_22:02:13_PDT_2023
Cuda compilation tools, release 12.2, V12.2.140
Build cuda_12.2.r12.2/compiler.33191640_0
Wed Nov 13 10:23:43 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   56C    P8              10W /  7


## Next, we write a native CUDA code and save it as 'vectorAdd.cu'


In [2]:
%%writefile vectorAdd.cu
#include <stdio.h>
#include <stdlib.h>
__global__ void add(int *a, int *b, int *c) {
*c = *a + *b;
}
int main() {
int a, b, c;
// host copies of variables a, b & c
int *d_a, *d_b, *d_c;
// device copies of variables a, b & c
int size = sizeof(int);
// Allocate space for device copies of a, b, c
cudaMalloc((void **)&d_a, size);
cudaMalloc((void **)&d_b, size);
cudaMalloc((void **)&d_c, size);
// Setup input values
c = 0;
a = 3;
b = 5;
// Copy inputs to device
cudaMemcpy(d_a, &a, size, cudaMemcpyHostToDevice);
  cudaMemcpy(d_b, &b, size, cudaMemcpyHostToDevice);
// Launch add() kernel on GPU
add<<<1,1>>>(d_a, d_b, d_c);
// Copy result back to host
cudaError err = cudaMemcpy(&c, d_c, size, cudaMemcpyDeviceToHost);
  if(err!=cudaSuccess) {
      printf("CUDA error copying to Host: %s\n", cudaGetErrorString(err));
  }
printf("result is %d\n",c);
// Cleanup
cudaFree(d_a);
cudaFree(d_b);
cudaFree(d_c);
return 0;
}

Writing vectorAdd.cu


In [3]:
%%writefile hw2_ex1.cu
#include <stdio.h>
#include <sys/time.h>
#include <test>
#include <>
#include test

#define DataType double

__global__ void vecAdd(DataType *in1, DataType *in2, DataType *out, int len) {
  //@@ Insert code to implement vector addition here
}

//@@ Insert code to implement timer start

//@@ Insert code to implement timer stop


int main(int argc, char **argv) {

  int inputLength;
  DataType *hostInput1;
  DataType *hostInput2;
  DataType *hostOutput;
  DataType *resultRef;
  DataType *deviceInput1;
  DataType *deviceInput2;
  DataType *deviceOutput;

  //@@ Insert code below to read in inputLength from args


  printf("The input length is %d\n", inputLength);

  //@@ Insert code below to allocate Host memory for input and output


  //@@ Insert code below to initialize hostInput1 and hostInput2 to random numbers, and create reference result in CPU


  //@@ Insert code below to allocate GPU memory here


  //@@ Insert code to below to Copy memory to the GPU here


  //@@ Initialize the 1D grid and block dimensions here


  //@@ Launch the GPU Kernel here


  //@@ Copy the GPU memory back to the CPU here


  //@@ Insert code below to compare the output with the reference


  //@@ Free the GPU memory here

  //@@ Free the CPU memory here

  return 0;
}

Writing hw2_ex1.cu


## We compile the saved cuda code using nvcc compiler

In [4]:
!nvcc vectorAdd.cu -o vectorAdd
!ls


hw2_ex1.cu  sample_data  vectorAdd  vectorAdd.cu


## Finally, we execute the binary of the compiled code

In [7]:
# !./vectorAdd
!nvprof --print-gpu-trace ./vectorAdd

==2715== NVPROF is profiling process 2715, command: ./vectorAdd
result is 8
==2715== Profiling application: ./vectorAdd
==2715== Profiling result:
   Start  Duration            Grid Size      Block Size     Regs*    SSMem*    DSMem*      Size  Throughput  SrcMemType  DstMemType           Device   Context    Stream  Name
168.96ms  1.1840us                    -               -         -         -         -        4B  3.2219MB/s    Pageable      Device     Tesla T4 (0)         1         7  [CUDA memcpy HtoD]
168.97ms     672ns                    -               -         -         -         -        4B  5.6766MB/s    Pageable      Device     Tesla T4 (0)         1         7  [CUDA memcpy HtoD]
169.18ms  3.3600us              (1 1 1)         (1 1 1)        16        0B        0B         -           -           -           -     Tesla T4 (0)         1         7  add(int*, int*, int*) [130]
169.19ms  2.0800us                    -               -         -         -         -        4B  1.834

In [9]:
!ncu ./vectorAdd

==PROF== Connected to process 3292 (/content/vectorAdd)
==PROF== Profiling "add(int *, int *, int *)" - 0: 0%....50%....100% - 8 passes
result is 8
==PROF== Disconnected from process 3292
[3292] vectorAdd@127.0.0.1
  add(int *, int *, int *) (1, 1, 1)x(1, 1, 1), Context 1, Stream 7, Device 0, CC 7.5
    Section: GPU Speed Of Light Throughput
    ----------------------- ------------- ------------
    Metric Name               Metric Unit Metric Value
    ----------------------- ------------- ------------
    DRAM Frequency          cycle/nsecond         4.63
    SM Frequency            cycle/usecond       546.22
    Elapsed Cycles                  cycle        1,993
    Memory Throughput                   %         0.54
    DRAM Throughput                     %         0.08
    Duration                      usecond         3.65
    L1/TEX Cache Throughput             %        19.98
    L2 Cache Throughput                 %         0.54
    SM Active Cycles                cycle        20